## Dependencies
Always make sure whatever environment you're working in has the necessary libraries and packages installed for your model to run. Here, I already pre-made a Docker container with all the necessary dependencies installed but in rl you'll have many alternatives, like virtual/conda environments, you can use for your specific project. What those things are is kinda out of the scope of this tutorial, so just take my word for granted lol

In [19]:
# ** RUN THIS BEFORE EVERYTHING! 
import numpy as np 
import pandas as pd 
from matplotlib import pyplot as plt
import sklearn
from sklearn.preprocessing import StandardScaler


## Data Exploration
I looked around the Internet to find a suitable dataset for this project. Ended up using a Kaggle dataset for classifying gene expression for types of leukemia!

Couple of options: 
1. Found a GEO entry here: https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE121711 as I went through open source papers.
The actual R script I wrote to get the file from the internet and process it is in the utils folder, but otherwise the actual preprocessing of the dataset is below. Personally, I couldn't get the accuracy past maybe 30% but feel free to try it out on your own!

2. There's a random Breast cancer dataset I found on Kaggle - also put it in the processed direcotory. 

In [2]:
# FUNCTION FOR LOADING IN THE DATA INTO NUMPY ARRAY: DO NOT TOUCH
def load_data(name, filepath): 
    
    # load in the CSV data into a pandas dataframe and briefly explore what it looks like 
    data = pd.read_csv(filepath, header=0)
    print("======================================================================")
    print(f"This is the initial shape/dimensions of the {name} data: ", data.shape)

    return data 

labels = load_data('label', '../data/kaggle/actual.csv')
train_data = load_data('training', '../data/kaggle/train.csv')
test_data = load_data('test', '../data/kaggle/test.csv')

def process_data(labels, train_data, test_data): 

    # remove superfluous columns from the train_data, test_data (call)
    train_data, test_data = train_data.loc[:, [col for col in train_data.columns if "call" not in col]], test_data.loc[:, [col for col in test_data.columns if "call" not in col]]

    # next remove the gene description, gene accession number, not absolutely necessary here 
    train_data, test_data = train_data.iloc[:,2:], test_data.iloc[:,2:]

    # next, binarize the labels
    binarized_labels = {"ALL": 0, "AML":1}
    labels["cancer"] = labels["cancer"].map(binarized_labels)
    # print(labels.tail(10))

    # next, set the train and test labels individually 
    train_labels = np.array([labels.iloc[int(patient)-1, 1] for patient in train_data.columns])
    test_labels = np.array([labels.iloc[int(patient)-1, 1] for patient in test_data.columns])

    # now extract the features by converting dfs to numpy arrays 
    train_features, test_features = np.array(train_data), np.array(test_data)

    # print out dimensions for checking
    print(train_features.shape, train_labels.shape, test_features.shape, test_labels.shape)

    return train_features, train_labels, test_features, test_labels

train_features, train_labels, test_features, test_labels = process_data(labels, train_data, test_data)

This is the initial shape/dimensions of the label data:  (72, 2)
This is the initial shape/dimensions of the training data:  (7129, 78)
This is the initial shape/dimensions of the test data:  (7129, 70)
(7129, 38) (38,) (7129, 34) (34,)


In [21]:
# APPLY MIN-MAX SCALING AND VIZ FEATURE DISTRIBUTION

def min_max_scale(features): 
    minimum, maximum = np.min(features), np.max(features)
    return (features - minimum) / (maximum - minimum) 

# train_features, testing_features = min_max_scale(train_features), min_max_scale(test_features)
scaler = StandardScaler()
train_features = scaler.fit_transform(train_features)
# test_features = scaler.transform(test_features)

def plot_feature_distributions(idx, title, dist_to_plot, n_bins): 
    fig, ax = plt.subplots(1, 2, tight_layout=True)
    ax[idx].hist(dist_to_plot, bins=n_bins)
    ax[idx].title(title)
    
# plot_feature_distributions(1, "Scaled Data", flattened, len(flattened))


Since the train and test data were already set for us, there's no need to set the splits. However for other datasets we're going to need to set the training and testing splits for the data after having loaded it into Numpy array format, so use this section as you will!

In [15]:
def set_splits(data, splits:dict): 

    # randomly shuffle the data before splitting it 
    np.random.shuffle(data)

    # now split it 
    num_rows, num_cols = data.shape 
    training_features, training_labels = data[:int(splits['training']*num_rows),1:].T, data[:int(splits['training']*num_rows),0].T
    print(training_labels)
    testing_features, testing_labels = data[:int(splits['testing']*num_rows),1:].T,  data[:int(splits['testing']*num_rows),0].T
    print(f'Shape of training features is {training_features.shape} and labels is {training_labels.shape}')
    print(f'Shape of testing features is {testing_features.shape} and labels is {testing_labels.shape}')
    return training_features, training_labels, testing_features, testing_labels 

# we're going to allocate 80% of the data for training, 20% for testing. The testing entry is technically not used but it's there for clarity 
# training_features, training_labels, testing_features, testing_labels = set_splits(data, {'training': 0.8, 'testing': 0.2})

## Model Construction 
As a recap, these are the dimensions of the model that we will use (training data) for any m training examples (here, it's 24 for training and 6 for testing). For i=1...m training examples, just add an extra dimension of the training examples to the matrices and multiply with that as a common internal dimension:
 
- X = (1000, m) 
- W1 = (1000, 500), B1 = (500, 1) {bias can be 1 dimensional here b/c of broadcasting!}, Z1 = (500, m), A1 = (500, m)
- W2 = (500, 100), B1 = (100, 1), Z2 = (100, m), A2 = (100, m)
- W3 = (1, 100), B1 = (1, 1), Z3 = (1, m), A3 = (1, m) => last layer, to which we apply sigmoid function!

Of note is that we only are using 2 layers in this network, which I believe is the practical amount for most tasks using linear neural networks (generally there's diminishing returns past 2-3 layers). Feel free to experiment and add more to this model though! 

Also, note that we don't define the loss function explicitly in the below model. This is because the choice of loss function is implicit in the formulas for the derivatives noted (we calculated the derivatives by hand earlier), but it's fairly straightforward to just use built-in libraries to compute the derivative. Thus the other parameters are pre-set as follows: 

- Optimizer: Vanilla Gradient Descent 
- Number epochs: 25 (change this as much as desired!)
- Learning rate: 0.001

In [35]:

# If this were a multi-class classification problem, we would need something like this function. Just use this for ur own info as you will! 
def one_hot_encode_labels(Y, number_of_classes): 
    # --------------------------------------------------------------------------------------------------------------
    # for every training example we're going to have a corresponding label
    # however because the softmax function wants to compute a PROBABILITY, we should one-hot-encode the labels to be from range 0-1 to better compute the loss
    # --------------------------------------------------------------------------------------------------------------
    one_hot_Y = np.zeros((Y.size, Y.max()+1))
    one_hot_Y[np.arange(Y.size), Y.astype(int)] = 1
    one_hot_Y = one_hot_Y.T
    return one_hot_Y

def relu_activation(Z): 
    return np.maximum(0, Z)

def relu_derivative(Z): 
    return Z < 0

def sigmoid_output(Z): 
    return 1 / (1 + np.exp(-Z))

def softmax_output(Z):
    return np.exp(Z) / sum(np.exp(Z))

def initialize_parameters(input_layer_units, layer_1_units, layer_2_units):
    
    # we need to initialize our weight and bias matrices here. subtract 0.5 for numerical stability 
    W1 = np.random.randn(layer_1_units, input_layer_units) - 0.5
    B1 = np.random.randn(layer_1_units, 1) 
    W2 = np.random.randn(layer_2_units, layer_1_units) - 0.5
    B2 = np.random.randn(layer_2_units, 1)
    
    return W1, B1, W2, B2

def forward_propagation(X, Y, W1, B1, W2, B2): 

    # print("------------------------------------")
    # Y = one_hot_encode_labels(Y, 6)
    Z1 = np.matmul(W1, X) + B1
    A1 = relu_activation(Z1)
    # print("First activation's shape: ", A1.shape)
    Z2 = np.matmul(W2, A1) + B2
    A2 = sigmoid_output(Z2)
    # print("Second activation's shape: ", A2.shape)
    # print("------------------------------------")

    return Y, Z1, A1, Z2, A2

def backward_propagation(num_examples, W2, X, Y, Z1, A1, A2): 

    dZ2 = A2 - Y
    # print("dZ2 shape: ", dZ2.shape)
    dW2 = (1 / num_examples) * np.matmul(dZ2, A1.T)
    # print("dW2 shape: ", dW2.shape)
    dB2 = (1 / num_examples) * np.sum(dZ2, axis=1, keepdims=True) 
    # print("dB2 shape: ", dB2.shape)
    dZ1 = relu_derivative(Z1) * np.matmul(W2.T, dZ2)
    # print("dZ1 shape: ", dZ1.shape)
    dW1 = (1 / num_examples) * np.matmul(dZ1, X.T)
    # print("dW1 shape: ", dW1.shape)
    dB1 = (1 / num_examples) * np.sum(dZ1, axis=1, keepdims=True) 
    # print("dB1 shape: ", dB1.shape)

    return dW2, dB2, dW1, dB1

def pred(A):
    # return np.array(A > 0.5, dtype='int')
    return np.argmax(A, 0)

def accuracy(pred, Y):
    accuracy = 0
    for i in range(pred.shape[1]):
        if (pred[0][i] > 0.5 and Y[i] == 1) or (pred[0][i] <= 0.5 and Y[i] == 0):
            accuracy += 1
    return accuracy / len(Y)

def gradient_descent(num_epochs, learning_rate, X, Y, W1, B1, W2, B2): 
     
    # train for the decided number of epochs (basically # of passes over the same training dataset)
    for epoch in range(num_epochs): 

        # make forward & backward pass to calculate loss
        Y, Z1, A1, Z2, A2 = forward_propagation(X, Y, W1, B1, W2, B2)
        dW2, dB2, dW1, dB1 = backward_propagation(34, W2, X, Y, Z1, A1, A2)
        
        # step - update params w/ gradient descent! 
        W1 = W1 - learning_rate * dW1
        W2 = W2 - learning_rate * dW2 
        B1 = B1 - learning_rate * dB1
        B2 = B2 - learning_rate * dB2

        # calculate accuracy metrics! 
        if epoch % 5 == 0: 
            print(f'Epoch number: {epoch} | Accuracy: {accuracy(A2, Y)}')
 
def train_model(X, Y): 

    # initialize parameters 
    num_epochs, learning_rate = 100, 0.001
    W1, B1, W2, B2 = initialize_parameters(7129, 5500, 1)
    gradient_descent(num_epochs, learning_rate, X, Y, W1, B1, W2, B2)

train_model(train_features, train_labels)

/var/folders/_c/xlv9fsg15_df99yllrpzxrw80000gn/T/ipykernel_60721/3882566409.py:19: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-Z))


Epoch number: 0 | Accuracy: 0.7105263157894737
Epoch number: 5 | Accuracy: 0.7105263157894737
Epoch number: 10 | Accuracy: 0.7105263157894737
Epoch number: 15 | Accuracy: 0.7105263157894737
Epoch number: 20 | Accuracy: 0.3684210526315789
Epoch number: 25 | Accuracy: 0.8421052631578947
Epoch number: 30 | Accuracy: 0.6052631578947368
Epoch number: 35 | Accuracy: 0.631578947368421
Epoch number: 40 | Accuracy: 0.7894736842105263
Epoch number: 45 | Accuracy: 0.8421052631578947
Epoch number: 50 | Accuracy: 0.9473684210526315
Epoch number: 55 | Accuracy: 0.9473684210526315
Epoch number: 60 | Accuracy: 0.9473684210526315
Epoch number: 65 | Accuracy: 0.9473684210526315
Epoch number: 70 | Accuracy: 0.9473684210526315
Epoch number: 75 | Accuracy: 0.9473684210526315
Epoch number: 80 | Accuracy: 0.9473684210526315
Epoch number: 85 | Accuracy: 0.9473684210526315
Epoch number: 90 | Accuracy: 0.9473684210526315
Epoch number: 95 | Accuracy: 0.9473684210526315
